In [1]:
import pandas as pd 
import numpy as np
import json
from gensim.models import word2vec
import jieba 
import jieba.analyse

## 用wiki中文corpus建立word2vec模型
1. 讀檔編碼, 取出content
2. 使用jieba切詞
3. 製作model

#### 讀檔
1. 讀檔
2. 繁簡轉換


In [2]:
with open('WikiJson.json', 'r',encoding='utf-8') as f:
    data = json.load(f)

wiki = pd.DataFrame(data)
wiki.head()

,content,id,title,url
0,数学\n数学是利用符号语言研究數量、结构、变化以及空间等概念的一門学科，从某种角度看屬於形式...,13,数学,https://zh.wikipedia.org/wiki?curid=13
1,哲学\n哲學（），可以是特指學術上針對思考方法的學問，也可以是指对普遍而基本的主題的研究，这...,18,哲学,https://zh.wikipedia.org/wiki?curid=18
2,文學\n文学是指以语言文字为工具形象化地反映艺术，包括戏剧、诗歌、小说、散文等，是文化的重要...,21,文學,https://zh.wikipedia.org/wiki?curid=21
3,历史\n歷史，或简称史，指人类社会过去的事件和行动，以及对这些事件行为有系统的记录、诠释和研...,22,历史,https://zh.wikipedia.org/wiki?curid=22
4,计算机科学\n计算机科学用于解决信息与计算的理论基础，以及实现和应用它们的实用技术。\n计算...,25,计算机科学,https://zh.wikipedia.org/wiki?curid=25


#### 繁簡轉換, 使用OpenCC
1. pip install opencc-python-reimplemented
2. OpenCC() 設定<br>
    t2s - 繁轉簡<br>
    s2t - 簡轉繁<br>
    mix2t - 混和轉繁<br>
    mix2s - 混和轉簡<br>
3. OpenCC().convert(x) 來轉換

In [3]:
from opencc import OpenCC

In [4]:
cc = OpenCC('s2t')

In [5]:
wiki['content'] = wiki['content'].apply(lambda x: cc.convert(x))
wiki['title'] = wiki['title'].apply(lambda x: cc.convert(x))

In [9]:
wiki.head()

,content,id,title,url
0,數學\n數學是利用符號語言研究數量、結構、變化以及空間等概念的一門學科，從某種角度看屬於形式...,13,數學,https://zh.wikipedia.org/wiki?curid=13
1,哲學\n哲學（），可以是特指學術上針對思考方法的學問，也可以是指對普遍而基本的主題的研究，這...,18,哲學,https://zh.wikipedia.org/wiki?curid=18
2,文學\n文學是指以語言文字爲工具形象化地反映藝術，包括戲劇、詩歌、小說、散文等，是文化的重要...,21,文學,https://zh.wikipedia.org/wiki?curid=21
3,歷史\n歷史，或簡稱史，指人類社會過去的事件和行動，以及對這些事件行爲有系統的記錄、詮釋和研...,22,歷史,https://zh.wikipedia.org/wiki?curid=22
4,計算機科學\n計算機科學用於解決信息與計算的理論基礎，以及實現和應用它們的實用技術。\n計算...,25,計算機科學,https://zh.wikipedia.org/wiki?curid=25


#### 使用jieba分詞
1. 使用jieba分詞
2. 去除特殊符號

In [10]:
import re

In [20]:
def jieba_tokenizer(text):
    words = jieba.cut(text)
    return ' '.join(words)

In [29]:
wiki['content_tokenized'] = wiki.loc[:,'content'].apply(lambda x: jieba_tokenizer(x))

In [45]:
wiki['content_tokenized'] = wiki.loc[:,'content_tokenized'].apply(lambda x: re.sub('[#！＝=、，「」（）。《》()【】：？－ “ ” \n \' "" . ]',' ',x))

In [46]:
wiki.head()

,content,id,title,url,content_tokenized
0,數學\n數學是利用符號語言研究數量、結構、變化以及空間等概念的一門學科，從某種角度看屬於形式...,13,數學,https://zh.wikipedia.org/wiki?curid=13,數學 數學 是 利用 符號 語言 研究 數量 結構 變化 以及 空間 等 概念 ...
1,哲學\n哲學（），可以是特指學術上針對思考方法的學問，也可以是指對普遍而基本的主題的研究，這...,18,哲學,https://zh.wikipedia.org/wiki?curid=18,哲學 哲學 可以 是 特指 學術 上 針對 思考 方法 的 學問 也 可...
2,文學\n文學是指以語言文字爲工具形象化地反映藝術，包括戲劇、詩歌、小說、散文等，是文化的重要...,21,文學,https://zh.wikipedia.org/wiki?curid=21,文學 文學 是 指 以 語言 文字 爲 工具 形象 化地 反映 藝術 包括 戲劇 ...
3,歷史\n歷史，或簡稱史，指人類社會過去的事件和行動，以及對這些事件行爲有系統的記錄、詮釋和研...,22,歷史,https://zh.wikipedia.org/wiki?curid=22,歷史 歷史 或 簡稱 史 指 人類 社會 過去 的 事件 和 行動 以及 對...
4,計算機科學\n計算機科學用於解決信息與計算的理論基礎，以及實現和應用它們的實用技術。\n計算...,25,計算機科學,https://zh.wikipedia.org/wiki?curid=25,計算機 科學 計算機 科學 用於 解決 信息 與 計算 的 理論 基礎 以及 實現 ...


In [47]:
wiki.to_csv('wiki_2.csv',encoding='utf-8',index=False)

#### 製作model

In [2]:
wiki = pd.read_csv('wiki.csv')

In [3]:
corpus = wiki['content_tokenized'].values.tolist()
corpus_list = list(map(lambda x: x.split(), corpus))

In [4]:
model = word2vec.Word2Vec(corpus_list, size=300, iter=7, sg=0, hs=0)

In [5]:
model.save('word2vec_wiki')

In [6]:
model = word2vec.Word2Vec.load('word2vec_wiki')

In [7]:
""" 測試
"""
similar_df = pd.DataFrame()
similar_words = pd.DataFrame(model.wv.most_similar('橄欖油', topn=20), columns=['word', 'cos'])
similar_df = pd.concat([similar_df, similar_words], axis=0)

In [8]:
similar_df

,word,cos
0,乳酪,0.747784
1,酸奶,0.747329
2,糖漿,0.744256
3,調味品,0.741621
4,棕櫚油,0.737466
5,黃油,0.736021
6,雪利酒,0.735575
7,牛油,0.735180
8,蜂蜜,0.733975
9,調味料,0.732359
